# Variable dependiente: Monto Corrupción Amplia (numérica)

## 1. Data

#### 1.1. Import libraries and data

In [1]:
import warnings
warnings.filterwarnings('ignore')

In [2]:
import pandas as  pd, numpy as np
import matplotlib.pyplot as plt
import statsmodels.api as sm
import statsmodels.formula.api as smf

In [3]:
import funciones as fun
import variables_nombres as vn

In [4]:
path = r'..\..\..\input\preprocessed_data\base1.csv'
data = pd.read_csv( path )

In [7]:
data[ vn.dependientes_variables ]

KeyError: "['monto_corrup1', 'monto_corrup2'] not in index"

In [18]:
# Borrar otras variables dependientes. Solo nos quedamos con 'monto_corrup2'
data = data.drop( [ 'corrup_intensa', 'corrup_amplia', 'per_corrup1', 'per_corrup2', 'tipo_control' ], 
                  axis = 1 )

In [19]:
data

,sexo,facebook,web,propie_muni,mue_pro_1,mue_pro_4,mue_pro_5,mue_pro_6,mue_pro_7,mue_pro_10,...,devppimtotfun_f5trab,tejgtotfun_f5trans,devppimtotfun_f5trans,devppimtotfun_f5turi,piagtotfun_f5viv,tejgtotfun_f5viv,tdvgtotfun_f5viv,dfgpimpiatotfun_f5viv,devppimtotfun_f5viv,monto_corrup2
0,1.0,0.0,1.0,1.0,1.0,2.0,0.0,0.0,2.0,2.0,...,0.0,9.884865,4.390733,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,15.851884
1,1.0,0.0,0.0,1.0,1.0,1.0,1.0,0.0,2.0,2.0,...,0.0,10.601672,3.608485,4.614656,0.000000,9.648660,9.648660,0.022878,4.230487,12.060607
2,2.0,0.0,0.0,1.0,1.0,2.0,0.0,0.0,2.0,2.0,...,0.0,10.073483,4.609581,0.000000,0.000000,9.159152,9.159152,0.009500,4.615121,15.537098
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.0,13.502599,3.734943,4.615121,0.000000,0.000000,0.000000,0.000000,0.000000,15.031823
4,2.0,1.0,0.0,0.0,1.0,1.0,2.0,0.0,1.0,0.0,...,0.0,0.000000,4.442232,0.000000,0.000000,0.000000,12.583653,0.659707,3.811629,12.206308
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
692,2.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,1.0,0.0,...,0.0,13.249864,4.615109,0.000000,13.759167,12.402012,12.402012,-0.701920,4.615106,19.670970
693,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.0,13.043647,4.330858,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,13.223206
694,2.0,1.0,1.0,1.0,1.0,1.0,5.0,0.0,1.0,1.0,...,0.0,12.693200,4.597938,0.000000,12.105517,12.350079,12.350079,0.050389,4.613931,0.000000
695,2.0,1.0,1.0,1.0,1.0,1.0,0.0,1.0,1.0,1.0,...,0.0,15.332858,4.397264,4.615121,0.000000,14.455608,14.455183,2.545754,4.323754,16.768403


In [20]:
# Borrar aquellas filas en las que la variable dependiente tenga missing
data = data.dropna( axis = 0, subset = [ 'monto_corrup2' ] )

In [21]:
data = data.dropna( axis = 1 )

#### 1.2. Split data into training and test set

In [22]:
np.random.seed( 1234 )
training = np.random.choice( data.index, size=int( len( data )*( 3/4 )), replace = False )

data_train = data.loc[ training, : ]
data_test = data.drop( training, axis = 0 )

## 2. Definir el modelo

In [23]:
import statsmodels.formula.api as smf
import patsy

#### 2.1. Construir modelos

In [14]:
# data_vars = data.columns.tolist()
# data_vars = pd.DataFrame( data_vars )
# data_vars.to_excel( r'..\..\..\extra\varslists\b1_mca.xlsx' )

In [24]:
%%time

formula_basic = "monto_corrup2 ~ tejgfun_ct05pgercon + dfgpimpiafun_ct05pgercon + devppimfun_ct05pgercon + dfgdevpiagfun_ct05opseg + devppimfun_ct05trab + tejgfun_ct05come + dfgpimpiafun_ct05come + dfgdevpiagfun_ct05turi + piagfun_ct05agro + pimgfun_ct05agro + tejgfun_ct05agro + devppimfun_ct05agro + pimgfun_ct05energia + tejgfun_ct05energia + dfgdevpiagfun_ct05energia + devppimfun_ct05energia + piagfun_ct05ind + pimgfun_ct05ind + dfgpimpiafun_ct05ind + devppimfun_ct05ind + tejgfun_ct05trans + dfgdevpiagfun_ct05trans + dfgpimpiafun_ct05amb + dfgdevpiagfun_ct05amb + pimgfun_ct05san + tejgfun_ct05san + dfgpimpiafun_ct05san + devppimfun_ct05san + dfgpimpiafun_ct05viv + devppimfun_ct05viv + devppimfun_ct05cydep + devppimfun_ct05edu + piagfun_ct05protsoc + tejgfun_ct05protsoc + devppimfun_ct05protsoc + dfgpimpiafun_ct05prevsoc + devppimfun_ct05dpub + piagfun_ct06pgercon + tejgfun_ct06pgercon + piagfun_ct06opseg + tejgfun_ct06opseg + tdvgfun_ct06opseg + dfgdevpiagfun_ct06opseg + tdvgfun_ct06trab + tejgfun_ct06come + dfgpimpiafun_ct06come + devppimfun_ct06come + devppimfun_ct06agro + pimgfun_ct06pesca + dfgpimpiafun_ct06pesca + devppimfun_ct06energia + devppimfun_ct06ind + dfgpimpiafun_ct06trans + dfgdevpiagfun_ct06trans + devppimfun_ct06trans + piagfun_ct06san + pimgfun_ct06san + dfgdevpiagfun_ct06san + devppimfun_ct06san + tejgfun_ct06viv + tdvgfun_ct06viv + dfgdevpiagfun_ct06viv + pimgfun_ct06salud + dfgpimpiafun_ct06salud + devppimfun_ct06salud + piagfun_ct06edu + dfgpimpiafun_ct06edu + devppimfun_ct06protsoc + tdvgft_rdet + dfgpimpiaft_rdet + dfgdevpiagft_rdet + devppimft_rdet + piagrb_reod + tejgrb_reod + tdvgrb_reod + devppimrb_reod + devppimrb_redr + tejgrb_rooc + devppimrb_rooc + piagrb_dotr + dfgdevpiagrb_fonc + devppimrb_fonc + devppimrb_impm + devppimrb_canr + devppimct_r00gstcr + tejgct_r19gstcr + tejgct_r13gstcr + tdvgct_r13gstcr + devppimct_r13gstcr + devppimct_r07gstcr + tejgct_r07gstcp + piagct_r07srdeu + dfgpimpiact_r08gstcr + dfgdevpiagct_r08gstcr + tejgct_r18gstcr + dfgdevpiagct_r18gstcr + devppimct_r18gstcr + pimgge_r00ct05pobso + dfgpimpiage_r00ct05pobso + devppimge_r00ct05popso + tejgge_r00ct05biser + devppimge_r00ct05biser + tejgge_r00ct05otgst + tdvgge_r00ct05otgst + devppimgge_r00ct05otgst + dfgdevpiagge_r00ct06dotra + devppimgge_r00ct06otgst + dfgpimpiagge_r00ct06acanf + dfgpimpiagge_r09ct05pobso + dfgdevpiagge_r09ct05pobso + devppimgge_r09ct05pobso + piagge_r09ct05popso + dfgdevpiagge_r09ct05popso + devppimgge_r09ct05popso + dfgpimpiagge_r09ct05biser + dfgdevpiagge_r09ct05biser + pimgge_r09ct05dotra + tdvgge_r09ct05dotra + devppimgge_r09ct05dotra + piagge_r09ct05otgst + tejgge_r09ct05otgst + dfgdevpiagge_r09ct05otgst + devppimgge_r09ct05otgst + pimgge_r09ct06dotra + dfgpimpiagge_r09ct06dotra + devppimgge_r09ct06dotra + piagge_r09ct06otgst + tejgge_r09ct06otgst + tdvgge_r09ct06otgst + devppimgge_r09ct06otgst + pimgge_r09ct06acanf + tejgge_r09ct06acanf + dfgdevpiagge_r09ct06acanf + devppimgge_r09ct06acanf + dfgdevpiagge_r09ct07sedpu + devppimgge_r09ct07sedpu + dfgpimpiagge_r19ct06dotra + dfgdevpiagge_r19ct06dotra + devppimgge_r19ct06dotra + dfgpimpiagge_r19ct06acanf + devppimgge_r13ct05pobso + tejgge_r13ct05popso + dfgpimpiagge_r13ct05popso + dfgdevpiagge_r13ct05popso + devppimgge_r13ct05popso + piagge_r13ct05dotra + pimgge_r13ct05dotra + tejgge_r13ct05dotra + pimgge_r13ct05otgst + tejgge_r13ct05otgst + dfgdevpiagge_r13ct05otgst + devppimgge_r13ct05otgst + dfgpimpiagge_r13ct06dotra + dfgdevpiagge_r13ct06dotra + tejgge_r13ct06acanf + piagge_r07ct05pobso + tejgge_r07ct05pobso + dfgpimpiagge_r07ct05pobso + dfgdevpiagge_r07ct05pobso + devppimgge_r07ct05pobso + tejgge_r07ct05popso + tdvgge_r07ct05popso + dfgpimpiagge_r07ct05popso + devppimgge_r07ct05popso + tejgge_r07ct05biser + tdvgge_r07ct05biser + dfgdevpiagge_r07ct05biser + devppimgge_r07ct05biser + pimgge_r07ct05dotra + tejgge_r07ct05dotra + dfgpimpiagge_r07ct05dotra + devppimgge_r07ct05dotra + tejgge_r07ct05otgst + tdvgge_r07ct05otgst + dfgpimpiagge_r07ct05otgst + devppimgge_r07ct05otgst + dfgpimpiage_r08ct05pobso + devppimge_r08ct05pobso + dfgdevpiagge_r08ct05popso + devppimge_r08ct05popso + tejgge_r08ct05dotra + tdvgge_r08ct05dotra + dfgdevpiagge_r08ct05dotra + devppimge_r08ct05dotra + piagge_r08ct05otgst + tejgge_r08ct05otgst + dfgpimpiagge_r08ct05otgst + devppimgge_r08ct05otgst + dfgpimpiagge_r08ct06acanf + devppimgge_r08ct06acanf + dfgdevpiagge_r08ct06acacf + piagge_r08ct07sedpu + tejgge_r18ct05pobso + devppimge_r18ct05pobso + dfgdevpiagge_r18ct05popso + devppimge_r18ct05popso + tejgge_r18ct05dotra + dfgpimpiage_r18ct05dotra + devppimge_r18ct05dotra + piagge_r18ct05otgst + pimgge_r18ct05otgst + dfgdevpiagge_r18ct05otgst + devppimgge_r18ct05otgst + dfgpimpiagge_r18ct06acacf + piagge_r18ct07sedpu + piagkft_reod + devppimgkft_reod + piagkft_redr + piagkft_rooc + devppimgkft_dotr + devppimgkft_rdet + dfgpimpiagkftr07_rdet + dfgdevpiagkftr07_rdet + devppimgkftr07_rdet + dfgdevpiagkftr08_rdet + tejgfun_f1ct05agro + tejgfun_f1ct05amb + devppimfun_f1ct05amb + tejgfun_f1ct05come + devppimfun_f1ct05come + tejgfun_f1ct05cydep + dfgdevpiagfun_f1ct05cydep + devppimfun_f1ct05cydep + devppimfun_f1ct05edu + tejgfun_f1ct05opseg + dfgpimpiafun_f1ct05opseg + devppimfun_f1ct05opseg + tdvgfun_f1ct05pgercon + dfgdevpiagfun_f1ct05pgercon + devppimfun_f1ct05pgercon + tejgfun_f1ct05salud + tdvgfun_f1ct05salud + devppimfun_f1ct05salud + tejgfun_f1ct05san + dfgdevpiagfun_f1ct05san + devppimfun_f1ct05san + piagfun_f1ct05trans + pimgfun_f1ct05trans + tejgfun_f1ct05trans + devppimfun_f1ct05trans + devppimfun_f1ct05turi + tejgfun_f1ct05viv + tdvgfun_f1ct05viv + devppimfun_f1ct05viv + tdvgfun_f1ct06agro + devppimfun_f1ct06agro + devppimfun_f1ct06amb + tejgfun_f1ct06come + devppimfun_f1ct06come + tejgfun_f1ct06cydep + dfgpimpiafun_f1ct06cydep + piagfun_f1ct06edu + dfgpimpiafun_f1ct06edu + devppimfun_f1ct06edu + devppimfun_f1ct06opseg + dfgdevpiagfun_f1ct06pgercon + devppimfun_f1ct06pgercon + tejgfun_f1ct06protsoc + dfgpimpiafun_f1ct06protsoc + devppimfun_f1ct06protsoc + devppimfun_f1ct06salud + dfgpimpiafun_f1ct06trans + devppimfun_f1ct06trans + dfgdevpiagfun_f1ct06viv + devppimfun_f1ct06viv + piagfun_f2ct05agro + pimgfun_f2ct05agro + dfgdevpiagfun_f2ct05agro + devppimfun_f2ct05agro + devppimfun_f2ct05amb + dfgpimpiafun_f2ct05cydep + devppimfun_f2ct05cydep + piagfun_f2ct05edu + pimgfun_f2ct05edu + dfgpimpiafun_f2ct05edu + devppimfun_f2ct05edu + tejgfun_f2ct05energia + devppimfun_f2ct05energia + dfgdevpiagfun_f2ct05opseg + devppimfun_f2ct05opseg + piagfun_f2ct05protsoc + tejgfun_f2ct05protsoc + dfgdevpiagfun_f2ct05protsoc + tejgfun_f2ct05salud + dfgdevpiagfun_f2ct05salud + devppimfun_f2ct05salud + piagfun_f2ct05san + pimgfun_f2ct05san + tejgfun_f2ct05san + dfgpimpiafun_f2ct05san + devppimfun_f2ct05san + tdvgfun_f2ct05trans + dfgdevpiagfun_f2ct05trans + devppimfun_f2ct05trans + devppimfun_f2ct05turi + tejgfun_f2ct05viv + tdvgfun_f2ct05viv + devppimfun_f2ct05viv + piagfun_f2ct06agro + tejgfun_f2ct06agro + dfgpimpiafun_f2ct06agro + devppimfun_f2ct06agro + piagfun_f2ct06amb + tejgfun_f2ct06amb + tdvgfun_f2ct06amb + dfgdevpiagfun_f2ct06amb + devppimfun_f2ct06amb + piagfun_f2ct06come + pimgfun_f2ct06come + tejgfun_f2ct06come + dfgpimpiafun_f2ct06come + dfgdevpiagfun_f2ct06come + devppimfun_f2ct06come + tejgfun_f2ct06comunica + dfgpimpiafun_f2ct06comunica + dfgdevpiagfun_f2ct06comunica + tejgfun_f2ct06cydep + dfgpimpiafun_f2ct06cydep + dfgdevpiagfun_f2ct06cydep + devppimfun_f2ct06cydep + devppimfun_f2ct06edu + piagfun_f2ct06energia + pimgfun_f2ct06energia + tdvgfun_f2ct06energia + devppimfun_f2ct06energia + pimgfun_f2ct06ind + dfgdevpiagfun_f2ct06ind + devppimfun_f2ct06ind + piagfun_f2ct06opseg + tejgfun_f2ct06opseg + devppimfun_f2ct06opseg + piagfun_f2ct06pgercon + pimgfun_f2ct06pgercon + tejgfun_f2ct06pgercon + dfgdevpiagfun_f2ct06pgercon + devppimfun_f2ct06pgercon + piagfun_f2ct06protsoc + pimgfun_f2ct06protsoc + tdvgfun_f2ct06protsoc + dfgdevpiagfun_f2ct06protsoc + devppimfun_f2ct06protsoc + piagfun_f2ct06salud + tdvgfun_f2ct06salud + dfgpimpiafun_f2ct06salud + devppimfun_f2ct06salud + piagfun_f2ct06san + tejgfun_f2ct06san + tdvgfun_f2ct06san + devppimfun_f2ct06san + pimgfun_f2ct06trans + dfgdevpiagfun_f2ct06trans + devppimfun_f2ct06trans + piagfun_f2ct06turi + pimgfun_f2ct06turi + tejgfun_f2ct06turi + tdvgfun_f2ct06turi + devppimfun_f2ct06turi + tejgfun_f2ct06viv + tdvgfun_f2ct06viv + devppimfun_f2ct06viv + devppimfun_f3ct05opseg + dfgpimpiafun_f3ct05pgercon + tdvgfun_f3ct06opseg + devppimfun_f3ct06pgercon + tejgfun_f3ct06san + dfgpimpiafun_f4ct05agro + devppimfun_f4ct05agro + tejgfun_f4ct05amb + devppimfun_f4ct05amb + tdvgfun_f4ct05come + dfgpimpiafun_f4ct05come + dfgpimpiafun_f4ct05cydep + devppimfun_f4ct05cydep + devppimfun_f4ct05edu + dfgpimpiafun_f4ct05opseg + devppimfun_f4ct05opseg + pimgfun_f4ct05pgercon + tdvgfun_f4ct05pgercon + tejgfun_f4ct05protsoc + dfgdevpiagfun_f4ct05protsoc + devppimfun_f4ct05protsoc + tejgfun_f4ct05salud + tdvgfun_f4ct05salud + devppimfun_f4ct05salud + dfgpimpiafun_f4ct05san + dfgdevpiagfun_f4ct05trab + dfgpimpiafun_f4ct05trans + devppimfun_f4ct05trans + tejgfun_f4ct05turi + dfgpimpiafun_f4ct05turi + devppimfun_f4ct05turi + tdvgfun_f4ct05viv + dfgpimpiafun_f4ct05viv + devppimfun_f4ct05viv + devppimfun_f4ct06agro + dfgpimpiafun_f4ct06amb + dfgdevpiagfun_f4ct06come + tejgfun_f4ct06cydep + dfgdevpiagfun_f4ct06cydep + devppimfun_f4ct06cydep + tejgfun_f4ct06edu + tejgfun_f4ct06opseg + devppimfun_f4ct06opseg + piagfun_f4ct06pgercon + tejgfun_f4ct06pgercon + devppimfun_f4ct06pgercon + dfgpimpiafun_f4ct06protsoc + devppimfun_f4ct06protsoc + devppimfun_f4ct06salud + devppimfun_f4ct06trans + dfgdevpiagfun_f4ct06turi + devppimfun_f4ct06turi + tejgfun_f4ct06viv + devppimfun_f4ct06viv + pimgfun_f5ct05agro + tejgfun_f5ct05agro + dfgdevpiagfun_f5ct05amb + devppimfun_f5ct05amb + piagfun_f5ct05come + tejgfun_f5ct05come + piagfun_f5ct05comunica + devppimfun_f5ct05comunica + dfgdevpiagfun_f5ct05cydep + tdvgfun_f5ct05edu + dfgdevpiagfun_f5ct05edu + tdvgfun_f5ct05energia + pimgfun_f5ct05ind + dfgpimpiafun_f5ct05ind + dfgdevpiagfun_f5ct05opseg + devppimfun_f5ct05opseg + dfgdevpiagfun_f5ct05pgercon + tejgfun_f5ct05prevsoc + tdvgfun_f5ct05prevsoc + devppimfun_f5ct05prevsoc + tejgfun_f5ct05protsoc + tdvgfun_f5ct05protsoc + dfgpimpiafun_f5ct05protsoc + devppimfun_f5ct05protsoc + piagfun_f5ct05salud + devppimfun_f5ct05salud + piagfun_f5ct05san + devppimfun_f5ct05trab + tdvgfun_f5ct05trans + devppimfun_f5ct05trans + piagfun_f5ct05turi + tdvgfun_f5ct05turi + dfgpimpiafun_f5ct05turi + devppimfun_f5ct05turi + piagfun_f5ct05viv + dfgdevpiagfun_f5ct06agro + devppimfun_f5ct06amb + dfgpimpiafun_f5ct06come + dfgdevpiagfun_f5ct06come + devppimfun_f5ct06cydep + devppimfun_f5ct06opseg + devppimfun_f5ct06pgercon + tejgfun_f5ct06trab + devppimfun_f5ct06trab + dfgdevpiagfun_f5ct06trans + tejgfun_f5ct06turi + dfgdevpiagfun_f5ct06turi + devppimfun_f5ct06turi + tejgfun_f5ct06viv + devppimfun_f5ct06viv + piagfun_f5pesca + piagfun_f5r07ct05agro + pimgfun_f5r07ct05agro + tejgfun_f5r07ct05agro + devppimfun_f5r07ct05agro + piagfun_f5r07ct05amb + tejgfun_f5r07ct05amb + dfgdevpiagfun_f5r07ct05amb + devppimfun_f5r07ct05amb + tejgfun_f5r07ct05come + tdvgfun_f5r07ct05come + dfgdevpiagfun_f5r07ct05come + devppimfun_f5r07ct05come + piagfun_f5r07ct05comunica + tejgfun_f5r07ct05comunica + dfgdevpiagfun_f5r07ct05comunica + devppimfun_f5r07ct05comunica + piagfun_f5r07ct05cydep + tejgfun_f5r07ct05cydep + dfgdevpiagfun_f5r07ct05cydep + devppimfun_f5r07ct05cydep + dfgdevpiagfun_f5r07ct05dpub + pimgfun_f5r07ct05edu + dfgpimpiafun_f5r07ct05edu + devppimfun_f5r07ct05edu + pimgfun_f5r07ct05energia + tdvgfun_f5r07ct05energia + dfgpimpiafun_f5r07ct05energia + devppimfun_f5r07ct05energia + piagfun_f5r07ct05ind + dfgpimpiafun_f5r07ct05ind + dfgdevpiagfun_f5r07ct05opseg + dfgdevpiagfun_f5r07ct05pgercon + piagfun_f5r07ct05protsoc + pimgfun_f5r07ct05protsoc + dfgpimpiafun_f5r07ct05protsoc + dfgdevpiagfun_f5r07ct05protsoc + devppimfun_f5r07ct05protsoc + tejgfun_f5r07ct05salud + tdvgfun_f5r07ct05salud + devppimfun_f5r07ct05salud + pimgfun_f5r07ct05san + tejgfun_f5r07ct05san + dfgpimpiafun_f5r07ct05san + devppimfun_f5r07ct05san + piagfun_f5r07ct05trans + tejgfun_f5r07ct05trans + tdvgfun_f5r07ct05trans + dfgdevpiagfun_f5r07ct05trans + devppimfun_f5r07ct05trans + piagfun_f5r07ct05turi + tdvgfun_f5r07ct05turi + dfgpimpiafun_f5r07ct05turi + tejgfun_f5r07ct05viv + tdvgfun_f5r07ct05viv + dfgpimpiafun_f5r07ct05viv + dfgdevpiagfun_f5r07ct05viv + devppimfun_f5r07ct05viv + dfgdevpiagfun_f5r07ct06agro + devppimfun_f5r07ct06agro + piagfun_f5r07ct06amb + pimgfun_f5r07ct06amb + tejgfun_f5r07ct06amb + devppimfun_f5r07ct06amb + pimgfun_f5r07ct06come + tejgfun_f5r07ct06come + tdvgfun_f5r07ct06come + dfgpimpiafun_f5r07ct06come + devppimfun_f5r07ct06come + devppimfun_f5r07ct06comunica + piagfun_f5r07ct06cydep + pimgfun_f5r07ct06cydep + tejgfun_f5r07ct06cydep + devppimfun_f5r07ct06cydep + pimgfun_f5r07ct06energia + tejgfun_f5r07ct06energia + piagfun_f5r07ct06ind + pimgfun_f5r07ct06ind + dfgpimpiafun_f5r07ct06ind + devppimfun_f5r07ct06ind + piagfun_f5r07ct06opseg + tejgfun_f5r07ct06opseg + devppimfun_f5r07ct06opseg + pimgfun_f5r07ct06pgercon + tejgfun_f5r07ct06pgercon + dfgpimpiafun_f5r07ct06pgercon + devppimfun_f5r07ct06pgercon + dfgdevpiagfun_f5r07ct06prevsoc + pimgfun_f5r07ct06protsoc + tejgfun_f5r07ct06protsoc + devppimfun_f5r07ct06protsoc + pimgfun_f5r07ct06salud + devppimfun_f5r07ct06salud + tejgfun_f5r07ct06san + devppimfun_f5r07ct06trab + piagfun_f5r07ct06trans + dfgpimpiafun_f5r07ct06trans + devppimfun_f5r07ct06trans + tejgfun_f5r07ct06turi + devppimfun_f5r07ct06turi + piagfun_f5r07ct06viv + devppimfun_f5r07ct06viv + pimgfun_f5r08ct05agro + tejgfun_f5r08ct05agro + dfgdevpiagfun_f5r08ct05agro + devppimfun_f5r08ct05agro + dfgpimpiafun_f5r08ct05comunica + piagfun_f5r08ct05cydep + devppimfun_f5r08ct05cydep + tdvgfun_f5r08ct05dpub + tejgfun_f5r08ct05edu + devppimfun_f5r08ct05edu + piagfun_f5r08ct05energia + tejgfun_f5r08ct05energia + dfgdevpiagfun_f5r08ct05energia + devppimfun_f5r08ct05energia + dfgpimpiafun_f5r08ct05pgercon + dfgpimpiafun_f5r08ct05protsoc + pimgfun_f5r08ct05salud + tejgfun_f5r08ct05salud + devppimfun_f5r08ct05salud + piagfun_f5r08ct05san + pimgfun_f5r08ct05san + tejgfun_f5r08ct05san + dfgpimpiafun_f5r08ct05san + devppimfun_f5r08ct05san + dfgpimpiafun_f5r08ct05trans + dfgdevpiagfun_f5r08ct05trans + devppimfun_f5r08ct05trans + devppimfun_f5r08ct05viv + piagfun_f5r08ct06agro + devppimfun_f5r08ct06agro + tejgfun_f5r08ct06amb + tdvgfun_f5r08ct06amb + dfgdevpiagfun_f5r08ct06amb + devppimfun_f5r08ct06amb + pimgfun_f5r08ct06come + tejgfun_f5r08ct06come + tdvgfun_f5r08ct06come + dfgpimpiafun_f5r08ct06come + devppimfun_f5r08ct06come + piagfun_f5r08ct06comunica + pimgfun_f5r08ct06comunica + devppimfun_f5r08ct06comunica + piagfun_f5r08ct06cydep + tejgfun_f5r08ct06cydep + dfgpimpiafun_f5r08ct06cydep + devppimfun_f5r08ct06cydep + piagfun_f5r08ct06edu + pimgfun_f5r08ct06edu + devppimfun_f5r08ct06edu + piagfun_f5r08ct06energia + pimgfun_f5r08ct06energia + devppimfun_f5r08ct06energia + piagfun_f5r08ct06opseg + tejgfun_f5r08ct06opseg + tdvgfun_f5r08ct06opseg + devppimfun_f5r08ct06opseg + piagfun_f5r08ct06pgercon + tejgfun_f5r08ct06pgercon + devppimfun_f5r08ct06pgercon + piagfun_f5r08ct06protsoc + dfgdevpiagfun_f5r08ct06protsoc + devppimfun_f5r08ct06protsoc + pimgfun_f5r08ct06salud + tejgfun_f5r08ct06salud + dfgpimpiafun_f5r08ct06salud + devppimfun_f5r08ct06salud + devppimfun_f5r08ct06san + dfgpimpiafun_f5r08ct06trab + pimgfun_f5r08ct06trans + devppimfun_f5r08ct06trans + piagfun_f5r08ct06turi + tejgfun_f5r08ct06turi + devppimfun_f5r08ct06turi + piagfun_f5r08ct06viv + tejgfun_f5r08ct06viv + dfgpimpiafun_f5r08ct06viv + devppimfun_f5r08ct06viv + piagfun_f5r18ct05agro + devppimfun_f5r18ct05agro + piagfun_f5r18ct05amb + tejgfun_f5r18ct05amb + tdvgfun_f5r18ct05amb + dfgdevpiagfun_f5r18ct05amb + devppimfun_f5r18ct05amb + piagfun_f5r18ct05come + tejgfun_f5r18ct05come + dfgpimpiafun_f5r18ct05come + devppimfun_f5r18ct05come + tdvgfun_f5r18ct05comunica + dfgdevpiagfun_f5r18ct05comunica + pimgfun_f5r18ct05cydep + dfgpimpiafun_f5r18ct05cydep + devppimfun_f5r18ct05cydep + piagfun_f5r18ct05edu + tdvgfun_f5r18ct05edu + dfgdevpiagfun_f5r18ct05edu + devppimfun_f5r18ct05edu + piagfun_f5r18ct05energia + pimgfun_f5r18ct05energia + devppimfun_f5r18ct05energia + dfgpimpiafun_f5r18ct05ind + devppimfun_f5r18ct05ind + piagfun_f5r18ct05opseg + tejgfun_f5r18ct05opseg + dfgdevpiagfun_f5r18ct05opseg + devppimfun_f5r18ct05opseg + devppimfun_f5r18ct05pesca + devppimfun_f5r18ct05pgercon + pimgfun_f5r18ct05prevsoc + devppimfun_f5r18ct05prevsoc + pimgfun_f5r18ct05protsoc + tejgfun_f5r18ct05protsoc + tdvgfun_f5r18ct05protsoc + dfgdevpiagfun_f5r18ct05protsoc + devppimfun_f5r18ct05protsoc + tejgfun_f5r18ct05salud + dfgdevpiagfun_f5r18ct05salud + devppimfun_f5r18ct05salud + piagfun_f5r18ct05san + devppimfun_f5r18ct05san + devppimfun_f5r18ct05trab + piagfun_f5r18ct05trans + tejgfun_f5r18ct05trans + devppimfun_f5r18ct05trans + piagfun_f5r18ct05turi + tejgfun_f5r18ct05turi + tdvgfun_f5r18ct05turi + dfgpimpiafun_f5r18ct05turi + devppimfun_f5r18ct05turi + piagfun_f5r18ct05viv + pimgfun_f5r18ct05viv + tejgfun_f5r18ct05viv + tdvgfun_f5r18ct05viv + devppimfun_f5r18ct05viv + dfgpimpiafun_f5r18ct06agro + devppimfun_f5r18ct06agro + devppimfun_f5r18ct06amb + piagfun_f5r18ct06come + devppimfun_f5r18ct06come + devppimfun_f5r18ct06comunica + devppimfun_f5r18ct06cydep + devppimfun_f5r18ct06opseg + devppimfun_f5r18ct06pgercon + piagfun_f5r18ct06prevsoc + devppimfun_f5r18ct06prevsoc + tejgfun_f5r18ct06protsoc + devppimfun_f5r18ct06protsoc + dfgdevpiagfun_f5r18ct06salud + devppimfun_f5r18ct06salud + dfgpimpiafun_f5r18ct06trab + devppimfun_f5r18ct06trab + devppimfun_f5r18ct06trans + pimgfun_f5r18ct06turi + tdvgfun_f5r18ct06turi + dfgpimpiafun_f5r18ct06turi + dfgdevpiagfun_f5r18ct06viv + devppimfun_f5r18ct06viv + piagtotfun_f1agro + tdvgtotfun_f1agro + devppimtotfun_f1agro + tejgtotfun_f1amb + devppimtotfun_f1cydep + piagtotfun_f1edu + tejgtotfun_f1edu + dfgdevpiagtotfun_f1energia + devppimtotfun_f1energia + piagtotfun_f1opseg + pimgtotfun_f1opseg + tdvgtotfun_f1opseg + piagtotfun_f1prevsoc + dfgdevpiagtotfun_f1prevsoc + devppimtotfun_f1prevsoc + dfgpimpiatotfun_f1protsoc + piagtotfun_f1salud + pimgtotfun_f1salud + tejgtotfun_f1salud + piagtotfun_f1san + pimgtotfun_f1san + tejgtotfun_f1san + devppimtotfun_f1san + dfgdevpiagtotfun_f1trab + devppimtotfun_f1trab + tejgtotfun_f1trans + tdvgtotfun_f1turi + devppimtotfun_f1turi + piagtotfun_f2agro + dfgdevpiagtotfun_f2agro + devppimtotfun_f2agro + dfgpimpiatotfun_f2amb + dfgdevpiagtotfun_f2amb + tdvgtotfun_f2come + dfgpimpiatotfun_f2come + dfgdevpiagtotfun_f2come + pimgtotfun_f2comunica + piagtotfun_f2cydep + pimgtotfun_f2cydep + tejgtotfun_f2cydep + dfgpimpiatotfun_f2cydep + dfgpimpiatotfun_f2dpub + piagtotfun_f2edu + tdvgtotfun_f2edu + devppimtotfun_f2edu + piagtotfun_f2energia + tejgtotfun_f2pgercon + dfgpimpiatotfun_f2pgercon + devppimtotfun_f2pgercon + dfgpimpiatotfun_f2prevsoc + devppimtotfun_f2prevsoc + pimgtotfun_f2protsoc + devppimtotfun_f2protsoc + pimgtotfun_f2salud + piagtotfun_f2san + tejgtotfun_f2san + tdvgtotfun_f2san + dfgpimpiatotfun_f2san + dfgdevpiagtotfun_f2san + devppimtotfun_f2san + devppimtotfun_f2trab + tejgtotfun_f2trans + dfgpimpiatotfun_f2trans + devppimtotfun_f2trans + pimgtotfun_f2turi + tejgtotfun_f2turi + dfgpimpiatotfun_f2turi + dfgdevpiagtotfun_f2turi + dfgdevpiagtotfun_f2viv + piagtotfun_f3agro + tejgtotfun_f3agro + dfgpimpiatotfun_f3agro + devppimtotfun_f3agro + tdvgtotfun_f3amb + devppimtotfun_f3amb + dfgpimpiatotfun_f3comunica + tejgtotfun_f3cydep + dfgpimpiatotfun_f3cydep + devppimtotfun_f3cydep + tejgtotfun_f3edu + dfgpimpiatotfun_f3edu + dfgdevpiagtotfun_f3edu + devppimtotfun_f3edu + tejgtotfun_f3energia + dfgpimpiatotfun_f3energia + devppimtotfun_f3energia + tejgtotfun_f3opseg + tdvgtotfun_f3opseg + dfgpimpiatotfun_f3opseg + devppimtotfun_f3opseg + piagtotfun_f3pgercon + tejgtotfun_f3pgercon + dfgpimpiatotfun_f3pgercon + devppimtotfun_f3pgercon + tejgtotfun_f3protsoc + dfgpimpiatotfun_f3protsoc + devppimtotfun_f3protsoc + tejgtotfun_f3salud + devppimtotfun_f3salud + piagtotfun_f3san + pimgtotfun_f3san + devppimtotfun_f3san + tdvgtotfun_f3trans + devppimtotfun_f3trans + devppimtotfun_f3turi + piagtotfun_f3viv + tejgtotfun_f3viv + dfgpimpiatotfun_f3viv + devppimtotfun_f3viv + tejgtotfun_f4agro + tdvgtotfun_f4agro + piagtotfun_f4amb + tejgtotfun_f4amb + dfgpimpiatotfun_f4amb + devppimtotfun_f4amb + devppimtotfun_f4come + dfgpimpiatotfun_f4cydep + devppimtotfun_f4cydep + dfgpimpiatotfun_f4edu + devppimtotfun_f4edu + tejgtotfun_f4energia + dfgpimpiatotfun_f4energia + dfgdevpiagtotfun_f4energia + devppimtotfun_f4energia + piagtotfun_f4pgercon + pimgtotfun_f4pgercon + tejgtotfun_f4pgercon + devppimtotfun_f4pgercon + tejgtotfun_f4protsoc + piagtotfun_f4salud + pimgtotfun_f4salud + devppimtotfun_f4salud + dfgpimpiatotfun_f4san + devppimtotfun_f4san + dfgdevpiagtotfun_f4trab + devppimtotfun_f4trab + dfgpimpiatotfun_f4turi + devppimtotfun_f4turi + devppimtotfun_f5agro + tejgtotfun_f5amb + tdvgtotfun_f5amb + dfgdevpiagtotfun_f5amb + devppimtotfun_f5amb + tdvgtotfun_f5come + dfgdevpiagtotfun_f5come + devppimtotfun_f5come + piagtotfun_f5comunica + pimgtotfun_f5comunica + piagtotfun_f5cydep + tejgtotfun_f5cydep + tdvgtotfun_f5cydep + devppimtotfun_f5cydep + piagtotfun_f5dpub + tejgtotfun_f5dpub + tdvgtotfun_f5dpub + devppimtotfun_f5dpub + pimgtotfun_f5edu + devppimtotfun_f5edu + piagtotfun_f5ind + pimgtotfun_f5ind + tdvgtotfun_f5ind + devppimtotfun_f5ind + piagtotfun_f5opseg + tejgtotfun_f5opseg + tdvgtotfun_f5opseg + dfgpimpiatotfun_f5opseg + devppimtotfun_f5opseg + devppimtotfun_f5pgercon + dfgdevpiagtotfun_f5prevsoc + piagtotfun_f5protsoc + pimgtotfun_f5protsoc + dfgpimpiatotfun_f5protsoc + devppimtotfun_f5protsoc + piagtotfun_f5r07agro + pimgtotfun_f5r07agro + tejgtotfun_f5r07agro + devppimtotfun_f5r07agro + dfgpimpiatotfun_f5r07amb + dfgdevpiagtotfun_f5r07come + piagtotfun_f5r07comunica + tejgtotfun_f5r07comunica + dfgdevpiagtotfun_f5r07comunica + devppimtotfun_f5r07cydep + pimgtotfun_f5r07dpub + tejgtotfun_f5r07dpub + dfgpimpiatotfun_f5r07dpub + devppimtotfun_f5r07dpub + piagtotfun_f5r07edu + pimgtotfun_f5r07edu + tejgtotfun_f5r07edu + dfgpimpiatotfun_f5r07edu + devppimtotfun_f5r07edu + piagtotfun_f5r07energia + dfgdevpiagtotfun_f5r07energia + devppimtotfun_f5r07energia + pimgtotfun_f5r07ind + dfgpimpiatotfun_f5r07ind + dfgdevpiagtotfun_f5r07ind + tejgtotfun_f5r07opseg + tdvgtotfun_f5r07opseg + dfgdevpiagtotfun_f5r07opseg + devppimtotfun_f5r07opseg + piagtotfun_f5r07pesca + tejgtotfun_f5r07pesca + tdvgtotfun_f5r07pesca + dfgpimpiatotfun_f5r07pesca + tejgtotfun_f5r07pgercon + tdvgtotfun_f5r07pgercon + dfgpimpiatotfun_f5r07pgercon + devppimtotfun_f5r07pgercon + dfgdevpiagtotfun_f5r07prevsoc + devppimtotfun_f5r07prevsoc + piagtotfun_f5r07protsoc + tejgtotfun_f5r07protsoc + tdvgtotfun_f5r07protsoc + dfgpimpiatotfun_f5r07protsoc + piagtotfun_f5r07salud + dfgpimpiatotfun_f5r07salud + pimgtotfun_f5r07san + dfgpimpiatotfun_f5r07san + devppimtotfun_f5r07san + pimgtotfun_f5r07trab + dfgdevpiagtotfun_f5r07trab + devppimtotfun_f5r07trab + piagtotfun_f5r07turi + pimgtotfun_f5r07turi + tejgtotfun_f5r07turi + tdvgtotfun_f5r07turi + dfgpimpiatotfun_f5r07turi + devppimtotfun_f5r07turi + pimgtotfun_f5r07viv + tejgtotfun_f5r07viv + dfgpimpiatotfun_f5r07viv + devppimtotfun_f5r07viv + piagtotfun_f5r08agro + pimgtotfun_f5r08agro + dfgdevpiagtotfun_f5r08agro + tejgtotfun_f5r08amb + dfgdevpiagtotfun_f5r08amb + devppimtotfun_f5r08amb + dfgdevpiagtotfun_f5r08come + devppimtotfun_f5r08come + dfgdevpiagtotfun_f5r08cydep + tdvgtotfun_f5r08dpub + dfgdevpiagtotfun_f5r08dpub + devppimtotfun_f5r08dpub + piagtotfun_f5r08edu + tdvgtotfun_f5r08edu + dfgpimpiatotfun_f5r08edu + devppimtotfun_f5r08edu + piagtotfun_f5r08energia + devppimtotfun_f5r08energia + dfgpimpiatotfun_f5r08ind + devppimtotfun_f5r08ind + piagtotfun_f5r08opseg + tejgtotfun_f5r08opseg + dfgdevpiagtotfun_f5r08opseg + devppimtotfun_f5r08opseg + dfgdevpiagtotfun_f5r08prevsoc + devppimtotfun_f5r08prevsoc + dfgdevpiagtotfun_f5r08protsoc + devppimtotfun_f5r08protsoc + piagtotfun_f5r08salud + dfgdevpiagtotfun_f5r08salud + piagtotfun_f5r08san + tejgtotfun_f5r08san + tdvgtotfun_f5r08san + dfgpimpiatotfun_f5r08san + dfgdevpiagtotfun_f5r08trab + devppimtotfun_f5r08trab + piagtotfun_f5r08trans + dfgpimpiatotfun_f5r08trans + devppimtotfun_f5r08turi + tejgtotfun_f5r08viv + dfgpimpiatotfun_f5r08viv + tejgtotfun_f5r18agro + devppimtotfun_f5r18agro + piagtotfun_f5r18amb + tejgtotfun_f5r18amb + devppimtotfun_f5r18amb + pimgtotfun_f5r18come + devppimtotfun_f5r18come + piagtotfun_f5r18cydep + pimgtotfun_f5r18cydep + dfgdevpiagtotfun_f5r18cydep + tejgtotfun_f5r18dpub + dfgdevpiagtotfun_f5r18dpub + devppimtotfun_f5r18dpub + dfgdevpiagtotfun_f5r18edu + dfgpimpiatotfun_f5r18ind + devppimtotfun_f5r18ind + dfgpimpiatotfun_f5r18just + dfgdevpiagtotfun_f5r18just + piagtotfun_f5r18opseg + tejgtotfun_f5r18opseg + devppimtotfun_f5r18opseg + tejgtotfun_f5r18pesca + devppimtotfun_f5r18pesca + dfgdevpiagtotfun_f5r18pgercon + devppimtotfun_f5r18pgercon + pimgtotfun_f5r18prevsoc + pimgtotfun_f5r18protsoc + devppimtotfun_f5r18protsoc + piagtotfun_f5r18salud + devppimtotfun_f5r18salud + devppimtotfun_f5r18trab + dfgpimpiatotfun_f5r18trans + dfgdevpiagtotfun_f5r18trans + devppimtotfun_f5r18turi + piagtotfun_f5r18viv + tejgtotfun_f5r18viv + dfgpimpiatotfun_f5r18viv + devppimtotfun_f5r18viv + tdvgtotfun_f5salud + devppimtotfun_f5salud + tdvgtotfun_f5san + dfgdevpiagtotfun_f5san + devppimtotfun_f5san + piagtotfun_f5trab + tejgtotfun_f5trab + tdvgtotfun_f5trab + devppimtotfun_f5trab + tejgtotfun_f5trans + devppimtotfun_f5trans + devppimtotfun_f5turi + piagtotfun_f5viv + tejgtotfun_f5viv + tdvgtotfun_f5viv + dfgpimpiatotfun_f5viv + devppimtotfun_f5viv"
y_basic_train, x_basic_train = patsy.dmatrices( formula_basic, data_train, return_type = 'dataframe' )
y_basic_test, x_basic_test = patsy.dmatrices( formula_basic, data_test, return_type = 'dataframe' )
p_basic = x_basic_train.shape[ 1 ]

Wall time: 4.54 s


#### 2.2. Generar variables dependientes

In [25]:
Y_train = data_train[ 'monto_corrup2' ]
Y_test = data_test[ 'monto_corrup2' ]

In [26]:
p_basic

1054

## 3. OLS

In [27]:
x_basic_train.head()

,Intercept,tejgfun_ct05pgercon,dfgpimpiafun_ct05pgercon,devppimfun_ct05pgercon,dfgdevpiagfun_ct05opseg,devppimfun_ct05trab,tejgfun_ct05come,dfgpimpiafun_ct05come,dfgdevpiagfun_ct05turi,piagfun_ct05agro,...,tdvgtotfun_f5trab,devppimtotfun_f5trab,tejgtotfun_f5trans,devppimtotfun_f5trans,devppimtotfun_f5turi,piagtotfun_f5viv,tejgtotfun_f5viv,tdvgtotfun_f5viv,dfgpimpiatotfun_f5viv,devppimtotfun_f5viv
339,1.0,15.386930,1.351333,4.289474,0.307406,0.0,0.0,0.0,0.00,0.000000,...,0.0,0.0,14.711309,4.399793,0.000000,0.000000,12.027408,12.027408,0.197000,4.453322
549,1.0,13.155292,0.146427,4.511438,0.000000,0.0,0.0,0.0,0.03,0.000000,...,0.0,0.0,10.787709,3.442439,4.615121,0.000000,0.000000,0.000000,0.000000,0.000000
255,1.0,0.000000,0.188744,4.526656,0.000000,0.0,0.0,0.0,0.00,0.000000,...,0.0,0.0,0.000000,0.000000,4.467280,12.611541,0.000000,11.725138,-0.050000,3.921102
83,1.0,0.000000,2.125175,4.291729,-0.171162,0.0,0.0,0.0,0.00,9.179984,...,0.0,0.0,0.000000,4.256066,0.000000,0.000000,0.000000,0.000000,0.022200,0.000000
673,1.0,0.000000,-1.846324,4.425671,-0.185236,0.0,0.0,0.0,0.00,0.000000,...,0.0,0.0,0.000000,4.483049,0.000000,0.000000,0.000000,13.844683,1.095282,4.553912


#### 3.1. Ajustar el modelo

In [28]:
fit_lm_basic = smf.ols( formula_basic, data = data_train ).fit()

#### 3.2. Calcular out-of-sample MSE y Standard Error

In [29]:
# Only MSE

yhat_lm_basic = fit_lm_basic.predict( data_test )
print("The mean squared error (MSE) using the basic model is equal to:", ( ( Y_test - yhat_lm_basic )**2 ).mean() )

The mean squared error (MSE) using the basic model is equal to: 113.16806554148081


In [30]:
# MSE and SE

resid_basic  = ( Y_test - yhat_lm_basic )**2

MSE_lm_basic = sm.OLS( resid_basic , np.ones( resid_basic.shape[ 0 ] ) ).fit().summary2().tables[ 1 ].iloc[ 0, 0:2 ]
MSE_lm_basic

Coef.       113.168066
Std.Err.     15.907454
Name: const, dtype: float64

#### 3.3. Calcular R Cuadrado out-of-sample

In [31]:
R2_lm_basic = 1 - ( MSE_lm_basic[ 0 ]/Y_test.var() )
print( f"The R^2 using the basic model is equal to: { R2_lm_basic }" )

The R^2 using the basic model is equal to: -2.794676516985482


## 4. Lasso, Ridge and Elastic Net

#### 4.1. Theoretical Lasso from hdm package

In [32]:
import hdmpy as hdm

In [36]:
fit_rlasso = hdm.rlasso( x_basic_train.to_numpy() , Y_train.to_numpy().reshape( Y_train.size , 1 ) , post = False )
fit_rlasso_post = hdm.rlasso( x_basic_train.to_numpy() , Y_train.to_numpy().reshape( Y_train.size , 1 ) , post = True )

In [39]:
# Getting mean of each variable
meanx = x_basic_test.mean( axis = 0 ).values.\
                        reshape( x_basic_test.shape[ 1 ] , 1 )

# Reducing the mean
new_x1 = x_basic_test.to_numpy() - \
                    (np.ones( ( x_basic_test.shape[ 0 ] , 1 ) ) @ meanx.T)

# Getting the significant variables
x1_est_rlasso = new_x1[ :, fit_rlasso.est['index'].iloc[:, 0].to_list()]

# Getting the coef. from significant variables
beta_rlasso = fit_rlasso.est['beta'].loc[ fit_rlasso.est['index'].\
                                     iloc[:, 0].to_list(), ].to_numpy()

# yhat
yhat_rlasso = (x1_est_rlasso @ beta_rlasso) + np.mean( Y_test.to_numpy() )
residuals_rlasso = Y_test.to_numpy().reshape( Y_test.to_numpy().size, 1)  - yhat_rlasso

In [40]:
# Getting mean of each variable
meanx = x_basic_test.mean( axis = 0 ).values.\
                        reshape(x_basic_test.shape[ 1 ] , 1 )

# Reducing the mean
new_x1 = x_basic_test.to_numpy() - \
                    (np.ones( (x_basic_test.shape[ 0 ] , 1 ) ) @ meanx.T)

# Getting the significant variables
x1_est_rlasso_post = new_x1[ :, fit_rlasso_post.est['index'].iloc[:, 0].to_list()]

# Getting the coef. from significant variables
beta_rlasso_post = fit_rlasso_post.est['beta'].loc[ fit_rlasso_post.est['index'].\
                                     iloc[:, 0].to_list(), ].to_numpy()

# yhat
yhat_rlasso_post = (x1_est_rlasso_post @ beta_rlasso_post) + np.mean( Y_test.to_numpy() )
residuals_rlasso_post = Y_test.to_numpy().reshape( Y_test.to_numpy().size, 1)  - yhat_rlasso_post

In [41]:
MSE_lasso = sm.OLS( ( residuals_rlasso )**2 , np.ones( yhat_rlasso.size )  ).fit().summary2().tables[1].round(3)
MSE_lasso_post = sm.OLS( ( residuals_rlasso_post )**2  , np.ones( yhat_rlasso_post.size )  ).fit().summary2().tables[1].round(3)

R2_lasso = 1 - MSE_lasso.iloc[0, 0]/ np.var( Y_test )
R2_lasso_post = 1 - MSE_lasso_post.iloc[0, 0]/ np.var( Y_test )

print( f"The R^2 using the basic model is equal to {R2_lasso},for lasso and {R2_lasso_post} for post-lasso")

The R^2 using the basic model is equal to 0.04355234275579334,for lasso and -0.015532185134332188 for post-lasso


#### 4.2. Cross Validated Lasso, Ridge and Elastic Net

In [42]:
from sklearn.linear_model import LassoCV, RidgeCV, ElasticNetCV

In [43]:
fit_lasso_cv = LassoCV( cv = 10, fit_intercept = True, normalize = False, random_state = 0 ).fit( x_basic_train, Y_train )
fit_ridge = RidgeCV( cv = 10, fit_intercept = True, normalize = False, scoring = None ).fit( x_basic_train, Y_train )
fit_elnet = ElasticNetCV( cv = 10, fit_intercept = True, normalize = False, random_state = 0 ).fit( x_basic_train, Y_train )

yhat_lasso_cv = fit_lasso_cv.predict( x_basic_test )
yhat_ridge = fit_ridge.predict( x_basic_test )
yhat_elnet = fit_elnet.predict( x_basic_test )

residual_lasso = ( yhat_lasso_cv - Y_test )**2
residual_ridge = ( yhat_ridge - Y_test )**2
residual_elnet = ( yhat_elnet - Y_test )**2

MSE_lasso_cv = sm.OLS( residual_lasso, np.ones( Y_test.size )).fit().summary2().tables[ 1 ].round( 3 )
MSE_ridge = sm.OLS( residual_ridge, np.ones( Y_test.size )).fit().summary2().tables[ 1 ].round( 3 )
MSE_elnet = sm.OLS( residual_elnet, np.ones( Y_test.size )).fit().summary2().tables[ 1 ].round( 3 )

R2_lasso_cv = 1 - MSE_lasso_cv.iloc[ 0, 0 ] / np.var( Y_test )
R2_ridge = 1 - MSE_ridge.iloc[ 0, 0 ]  / np.var( Y_test )
R2_elnet = 1 - MSE_elnet.iloc[ 0, 0 ]  / np.var( Y_test )

print( "R^2 using cross-validation for lasso, ridge, and elastic net in the basic model: {:.5f}, {:.5f}, {:.5f}".format( R2_lasso_cv, R2_ridge, R2_elnet ) )

R^2 using cross-validation for lasso, ridge, and elastic net in the basic model: 0.02706, -2.19738, 0.02622


## 5. Non Linear Models

#### 5.1. Regression Trees

In [44]:
from sklearn.tree import DecisionTreeRegressor
from sklearn import tree

In [45]:
# Fit the model

fit_trees = DecisionTreeRegressor( random_state = 0, min_impurity_decrease = 0.001 )
fit_trees.fit( x_basic_train, y_basic_train )

DecisionTreeRegressor(min_impurity_decrease=0.001, random_state=0)

In [ ]:
# Plot the tree

# from sklearn.tree import plot_tree

# plt.figure( figsize=( 30, 20 ) )
# plot_tree( fit_trees, filled = True )
# plt.show()

In [46]:
# Determine the optimar complexity of the regression tree

s = pd.DataFrame( fit_trees.cost_complexity_pruning_path( y_basic_train, x_basic_train ) )
s.head()

,ccp_alphas,impurities
0,0.000000,1.467051
1,0.004601,1.471652
2,0.005304,1.476956
3,0.005389,1.482345
4,0.005523,1.487868


In [ ]:
### PREGUNTA: DE DÓNDE SALE ESE VALOR DE ALPHA

In [47]:
# Prune the tree

fit_prunnedtree = DecisionTreeRegressor( ccp_alpha = 0.00188444410871555 )
fit_prunnedtree.fit( x_basic_train, y_basic_train )

DecisionTreeRegressor(ccp_alpha=0.00188444410871555)

In [ ]:
# Plot the prunned tree

# plot_tree(fit_prunnedtree, filled=True)
# plt.show()

In [48]:
# Calculate MSE and R2 for prunned tree

y_hat_pt = fit_prunnedtree.predict( x_basic_test )
residual_pt = ( y_hat_pt - Y_test )**2
MSE_pt = sm.OLS( residual_pt, np.ones( y_hat_pt.size )).fit().summary2().tables[ 1 ].round( 3 )
R2_pt = 1 - MSE_pt.iloc[ 0, 0 ]/np.var( Y_test )
print( f"R^2 of the pruned tree: { R2_pt }" )

R^2 of the pruned tree: -0.8148932379765956


#### 5.2. Random Forest and Booested Trees

In [49]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import GradientBoostingRegressor 

In [50]:
# random forest
fit_rf = RandomForestRegressor( n_estimators = 2000, min_samples_leaf = 5).fit( x_basic_train, Y_train )

# boosting
fit_boost = GradientBoostingRegressor( loss = 'ls', learning_rate = 0.01, n_estimators = 1000, max_depth = 2, subsample = 0.5).fit( x_basic_train, Y_train )

# Evaluating the methods
yhat_rf = fit_rf.predict( x_basic_test )
residual_rf = ( yhat_rf - Y_test )**2
yhat_boost = fit_boost.predict( x_basic_test )
residual_bst = ( yhat_boost - Y_test )**2

# Calculate MSE
MSE_rf = sm.OLS( residual_pt, np.ones( yhat_rf.size )).fit().summary2().tables[ 1 ].round( 3 )
MSE_bst = sm.OLS( residual_bst, np.ones( yhat_boost.size )).fit().summary2().tables[ 1 ].round( 3 )

# Calculate R2
R2_rf = 1 - MSE_rf.iloc[ 0, 0 ] / Y_test.var()
R2_boost = 1 - MSE_bst.iloc[ 0, 0 ] / Y_test.var()

In [51]:
print("R^2 of the random forest and boosted trees:{:.5f}, {:.5f}".format( R2_rf, R2_boost ))

R^2 of the random forest and boosted trees:-0.80452, 0.03554


## 6. Resultados

In [52]:
table = pd.DataFrame(columns=["MSE", "S.E for MSE", "R-squared"]) 
table.loc[0]  = [MSE_lm_basic[0], MSE_lm_basic[1], R2_lm_basic]
table.loc[1]  = [MSE_lasso.iloc[0, 0], MSE_lasso.iloc[0, 1], R2_lasso]
table.loc[2]  = [MSE_lasso.iloc[0, 0], MSE_lasso_post.iloc[0, 1], R2_lasso_post]
table.loc[3]  = [MSE_lasso_cv.iloc[0, 0], MSE_lasso_cv.iloc[0, 1], R2_lasso_cv]
table.loc[4]  = [MSE_ridge.iloc[0, 0], MSE_ridge.iloc[0, 1], R2_ridge]
table.loc[5]  = [MSE_elnet.iloc[0, 0], MSE_elnet.iloc[0, 1], R2_elnet]
table.loc[6] = [MSE_rf.iloc[0, 0], MSE_rf.iloc[0, 1], R2_rf]
table.loc[7] = [MSE_bst.iloc[0, 0], MSE_bst.iloc[0, 1], R2_boost]
table.loc[8] = [MSE_pt.iloc[0, 0], MSE_pt.iloc[0, 1], R2_pt]
models_row = [ "Least Squares (basic)", "Lasso", "Post_Lasso", "Cross-Validated lasso", 
               "Cross-Validated ridge", "Cross-Validated elnet", "Random Forest", 
               "Boosted Trees", "Pruned Tree" ]
table.insert( 0, "Models", models_row )
table

,Models,MSE,S.E for MSE,R-squared
0,Least Squares (basic),113.168066,15.907454,-2.794677
1,Lasso,28.361000,3.373000,0.043552
2,Post_Lasso,28.361000,3.582000,-0.015532
3,Cross-Validated lasso,28.850000,3.582000,0.027061
4,Cross-Validated ridge,94.810000,13.006000,-2.197377
5,Cross-Validated elnet,28.875000,3.587000,0.026218
6,Random Forest,53.816000,6.490000,-0.804522
7,Boosted Trees,28.763000,3.532000,0.035538
8,Pruned Tree,53.816000,6.490000,-0.814893


In [ ]:
# Tabla general para Test
table.to_excel( r'..\..\..\output\ejecucion_2\results\base1_mca_results_training.xlsx' )